In [13]:
import sqlite3
from contextlib import closing

import pandas as pd

DB_PATH = 'MMAuto[GF260112](12-01-26-090617).mmbak'

# with sqlite.connect() -> auto-closes transaction but DOES NOT auto-close connection
# that's why also called contextlib.connect()
# https://blog.rtwilson.com/a-python-sqlite3-context-manager-gotcha/
# with closing(sqlite3.connect(DB_PATH)) as connection:   
#     pass  

connection = sqlite3.connect(DB_PATH)
# cursor = connection.cursor()

In [14]:
query = """
select case c.TYPE when 0 then 'Income' when 1 then 'Expense' end as type,
--       printf('%,d', sum(t.AMOUNT_ACCOUNT)) as total_amount
       sum(t.AMOUNT_ACCOUNT) as total_amount
from INOUTCOME t
join ZCATEGORY c on t.ctgUid = c.uid   -- inner join auto-excludes t.ctgUid=null (Transfer)
where t.IS_DEL = 0 and strftime('%Y-%m', t.WDATE) = '2025-12'
group by c.TYPE
"""
total_income_and_expense = pd.read_sql(query, connection)
total_income_and_expense       # BUG: wrong income, expenses figures are showing (comparing with actual stats shown in my MoneyManager app)

,type,total_amount
0,Income,78854.33
1,Expense,255768.73


In [43]:
query = """
select datetime(t.UTIME / 1000, 'unixepoch') as dt,
       case DO_TYPE
            when 3 then 'Transfer (from)'
            when 4 then 'Transfer (to)'
            else DO_TYPE
       end as do_type,
       case
            when t.ctgUid is null then 'Transfer'
            when t.ctgUid like '-%' or subcategory.TYPE = 1 then 'Expense'   -- (Modified Balance) ctgUid negative (seems to be -4 only)
            when subcategory.TYPE = 0 then 'Income' 
       end as type,
       case 
            when t.ctgUid is null then 'Transfer'
            when t.ctgUid like '-%' then 'Modified Balance'    -- ctgUid negative (seems to be -4 only)
            when category.NAME is null then subcategory.NAME 
            else printf('%s/%s', category.NAME, subcategory.NAME)
        end as category,
 --      c.SYMBOL || ' ' || t.AMOUNT_ACCOUNT as amount,
       printf('%s %,d', c.SYMBOL, t.AMOUNT_ACCOUNT) as amount,
       case
            when t.ctgUid is null then printf('%s -> %s', assets.NIC_NAME, toAssets.NIC_NAME)
            else assets.NIC_NAME 
       end as asset,
       t.ZCONTENT as name, 
       t.ZDATA as description
from INOUTCOME t
left join ZCATEGORY subcategory on subcategory.uid = t.ctgUid
left join ZCATEGORY category on category.uid = subcategory.pUid
join assets on assets.uid = t.assetUid
left join ASSETS toAssets on toAssets.uid = t.toAssetUid
join CURRENCY c on c.uid = t.currencyUid
where t.IS_DEL = 0 
      and t.DO_TYPE != 4   -- not Transfer (from) [because its duplicate Transfer (to) entry is already present]
      and t.ctgUid != ''   -- exclude Modified Balance entries where selected not to save as income/expense
order by t.UTIME desc
"""
# ZMONEY , IN_ZMONEY columns also seem to have exactly same values as AMOUNT_ACCOUNT - not sure if any difference
transactions = pd.read_sql(query, connection)
transactions

,dt,do_type,type,category,amount,asset,name,description
0,2026-01-11 11:47:48,1,Expense,Maintainence/Scooty,₹ 100,Cash,Mom Scooty Duplicate key,
1,2026-01-11 11:32:28,1,Expense,Household/Kitchen,₹ 65,Cash,"Bananas, Guava",
2,2026-01-11 11:25:28,1,Expense,Household/Kitchen,₹ 150,Cash,Khajoor,
3,2026-01-11 11:09:32,1,Expense,Food/Snacks & Beverages,₹ 5,Cash,sweets,
4,2026-01-11 04:19:34,1,Expense,Health,₹ 5,Cash,Saran ashram dentist filling removal,
...,...,...,...,...,...,...,...,...
2233,2022-05-23 18:11:58,0,Expense,Modified Balance,"₹ 43,848",ICICI Salary Account,Difference,Salary Account\n
2234,2022-05-23 15:38:42,7,Expense,Modified Balance,"₹ 2,977",UCO Savings Account,Difference,Linked to UPI
2235,2022-05-23 15:34:51,7,Expense,Modified Balance,₹ 725,Cash,Difference,in my wallet\n
2236,2022-05-23 15:30:49,7,Expense,Modified Balance,₹ 162,Paytm Wallet,Difference,


In [4]:
query = """
select budgetUid, AMOUNT
from BUDGET_AMOUNT
where IS_DEL = 0
"""
budget = pd.read_sql(query, connection)
print(budget.shape)
budget.head()

(58, 2)


,budgetUid,AMOUNT
0,9_1_0_0_6,1500.0
1,6_1_0_0_6,0.0
2,11_1_0_0_6,200.0
3,12_1_0_0_6,100.0
4,14_1_0_0_6,500.0


In [ ]:
query = """
select g.ACC_GROUP_NAME as accountGroup,
       t.NIC_NAME as name,
       t.ZDATA1 as description
from ASSETS t
join ASSETGROUP g on g.uid = t.groupUid
where t.ZDATA = 0
order by t.ORDERSEQ
"""
# here ZDATA=0 seems to mean not deleted, ZDATA=1 -> deleted
assets = pd.read_sql(query, connection)
assets

,accountGroup,name,description
0,Emergency Fund,Emergency Fund,Goal: 10L for car or flat\n-------- OLD ------...
1,Savings,Cash,"3000 in bag, rest in wallet"
2,Savings,UCO Savings Account,09500110036105\nUPI: sohangchopra-1@okhdfcbank...
3,Savings,ICICI Salary Account,008701053026\nUPI: sohangchopra-1@okicici (GPa...
4,Top-Up/Prepaid,Metro Card,
5,Investments,Quant Small Cap Fund,
6,Investments,Zerodha Kite Stocks,Discount Broker\nNeed to use instead of ICICI ...
7,Retirement,ICICI PPF,000418476401\nMatures in 15 years (2037)
8,Retirement,Employee PF,DSNHP16818210000010262\nMonthly:\nEmployee Sha...


In [18]:
query = """
select category.NAME as category,
       fromAssets.NIC_NAME as fromAsset,
       toAssets.NIC_NAME as toAsset,
       c.SYMBOL || ' ' || t.AMOUNT_SUB as amount,
       t.PAYEE as name,
       t.MEMO as description
from REPEATTRANSACTION t
left join ZCATEGORY category on category.uid = t.ctgUid
join ASSETS fromAssets on fromAssets.uid = t.assetUid
left join ASSETS toAssets on toAssets.uid = t.toAssetUid
join CURRENCY c on c.uid = t.currencyUid
where t.IS_DEL = 0
"""
repeat_transactions = pd.read_sql(query, connection)
repeat_transactions

,category,fromAsset,toAsset,amount,name,description
0,Salary,Employee PF,None,₹ 5000.0,PF by HCL (Sankalp Semiconductor Ltd),
1,None,ICICI PPF,ICICI Salary Account,₹ 500.0,PPF,Minimum Contribution
2,Donation,UCO Savings Account,None,₹ 3000.0,Newslaundry Annual Subscription,AutoPay is setup via GPay till March 2033\nCan...
3,Donation,ICICI Salary Account,None,₹ 3090.0,Educate Girls NGO,"Autopay setup in GPay UPI, \nCan cancel from t..."
4,Other,ICICI Salary Account,None,₹ 3000.0,Boring Money Annual Subscription,
5,Donation,ICICI Salary Account,None,₹ 119.0,Kurzgesagt: Paid Membership: youtube,Via GPay monthly autopay
6,Donation,ICICI Salary Account,None,₹ 2950.0,To Scroll.in news,via GPay AutoPay
7,Donation,ICICI Salary Account,None,₹ 59.0,Sarthak Goswami Youtube channel subscription,


In [21]:
query = """
select memoDate, title, content
from MEMO
where isDel = 0
order by orderSeq
"""
notes = pd.read_sql(query, connection)
notes

,memoDate,title,content
0,2023-06-28,Noida Stay126 PG,"Mom paid Rs. 32,500 (2 months advance, 1 month..."
1,2023-11-06,56k: Navi -> Quant,\n
2,2023-11-05,,20k RD transfer got missed due to less funds i...
3,2023-12-25,Chyawanprash Seva,Monday-Saturday: 25-30 December\n9 a.m. - ~4:1...
4,2024-02-11,Went to Parakh Wedding Satsang,\n
5,2024-04-07,"Phone Damaged, repl with Dad phone next Agra trip",\n
6,2024-06-23,Met Rishang @ Bhagat Halwai,\n
7,2024-06-28,Changed Toothbrush,\n
8,2024-08-06,IRCTC Ticket Refund,"Vande Bharat : received Rs. 1.5K, charge: Rs. ..."
9,2024-10-05,Left Noida PG,Monu (Stay126 PG owner) didn't refund Rs. 15K ...
